## Converting IEX trades to CSV

This notebook requires the [IEXTools](https://pypi.org/project/IEXTools/) library (available on PyPI via pip) and the TOPS pcap file for 2019-10-21 and 2019-10-25 available from the [IEX exchange](https://iextrading.com/trading/market-data/#hist-download).

In [ ]:
import pandas as pd
from IEXTools import Parser, messages #noqa Version 0.0.7

### Load parser for pcap file


In [ ]:
date = '2019-10-21' # Note that this notebook needs to be run once per date to process'
p = Parser(rf'{date}.pcap', tops=True, deep=False)

### Collect trade reports

In [ ]:
allowed = [messages.TradeReport]
messages = []
i = 0
while True:
    try:
        message = p.get_next_message(allowed)
        messages.append(message)
        i += 1
    except StopIteration:
        break

## Build a `DataFrame`

In [ ]:
fields = ['symbol', 'size', 'price_int', 'timestamp']
df = pd.DataFrame([{f:getattr(m, f) for f in fields} for m in messages])

## Convert `price_int` to `price`

In [ ]:
df['price_int'] = df['price_int'] / 10000.0
df = df.rename(columns={'price_int':'price'})
df.head()

### Save to CSV

In [ ]:
df.to_csv(f'IEX_{date}.csv', index=False)